In [ ]:
# ТЕСТИРОВАНИЕ A10 RELEASE

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf /content/drive/MyDrive/KIA_TEST/FAISS_KIA_V3

In [ ]:
!mkdir -p /content/drive/MyDrive/KIA_TEST/FAISS_KIA_V3

In [ ]:
!git clone https://github.com/musicnova/KIA-GPT0.git /content/drive/MyDrive/KIA_TEST/FAISS_KIA_V3/faiss_a10

Cloning into '/content/drive/MyDrive/KIA_TEST/FAISS_KIA_V3/faiss_a10'...
remote: Enumerating objects: 1558, done.
remote: Counting objects: 100% (372/372), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 1558 (delta 274), reused 281 (delta 227), pack-reused 1186
Receiving objects: 100% (1558/1558), 20.03 MiB | 13.44 MiB/s, done.
Resolving deltas: 100% (890/890), done.
Updating files: 100% (143/143), done.


In [ ]:
!(cd /content/drive/MyDrive/KIA_TEST/FAISS_KIA_V3/faiss_a10 && git checkout faiss/A10)

Branch 'faiss/A10' set up to track remote branch 'faiss/A10' from 'origin'.
Switched to a new branch 'faiss/A10'


In [ ]:
!rm -rf faiss

In [ ]:
!rm -f database.md

In [ ]:
!cp -r /content/drive/MyDrive/KIA_TEST/FAISS_KIA_V3/faiss_a10/knowledge/faiss faiss

In [ ]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13


## Собрать новую базу FAISS

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
import re
from langchain.vectorstores import FAISS

def load_file_knowledge(file_path: str) -> str:
    # Чтение текстового файла
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(text)

    # Предполагается, что FAISS и OpenAIEmbeddings были импортированы или определены где-то выше
    vectordateBase = FAISS.from_documents(md_header_splits, OpenAIEmbeddings())
    vectordateBase.save_local("faiss")

    return



In [ ]:
# load_file_knowledge('database.md')
# БД сохранитсья в коллабе в папке faiss нужно подождать какое-то время

## Проверка чанков **БЕЗ** сбора нового файла FAISS **БЕЗ** обращения к OpenAI


In [ ]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8

In [ ]:
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
#database
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.document_loaders import TextLoader

from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


HISTORY = []

In [ ]:
MODEL_GPT_3_5_TURBO_16K = ['gpt-3.5-turbo-16k', 0.003, 0.004]
MODEL_GPT_3_5_TURBO = ['gpt-3.5-turbo', 0.0015, 0.002]  # 4,097 tokens
MODEL_GPT_4 = ['gpt-4', 0.03, 0.06]  # 8,192 tokens
SELECT_MODEL_GPT = MODEL_GPT_3_5_TURBO_16K

In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
# Загружаем векторный файл базы знаний! В коллаб надо в папку faiss загрузить 2 файла

knowledge_only_langchain = FAISS.load_local("faiss", OpenAIEmbeddings())

In [ ]:
def insert_newlines(textstr: str, max_len: int = 170) -> str:
    words = textstr.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
def _summarize_topic( dialog):
        messages = [
            {"role": "system",
             "content": "Ты - ассистент консультанта, основанный на AI. Ты умеешь профессионально суммаризировать присланные тебе диалоги консультанта и клиента. Твоя задача - суммаризировать диалог, который тебе пришел."},
            {"role": "user",
             "content": "Суммаризируй следующий диалог консультанта и клиента: " + " ".join(dialog)}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=0.1,  # Используем более низкую температуру для более определенной суммаризации
            max_tokens=1000  # Ограничиваем количество токенов для суммаризации
        )

        return completion.choices[0].message.content



def answer_index(knowledge_base, topic, temp=0.1, top_similar_documents=5):
        summarize_text = ''
        if len(HISTORY) > 0:
            summarize_text = "Вот краткий обзор предыдущего диалога: " + _summarize_topic(
                [q + ' ' + (a if a is not None else '') for q, a in HISTORY])
            print(f'САММАРИ \n=== {summarize_text} \n')

        # Добавляем явное разделение между историей диалога и текущим вопросом
        input_text = summarize_text + "\n\nТекущий вопрос: " + topic

        docs = knowledge_base.similarity_search_with_score(topic, k=top_similar_documents)
        responses = []
        for i, (doc, score) in enumerate(docs):
            if score < 1: # ТУТ ТЫ МОЖЕШЬ УПРАВЛЯТЬ праметром Л2 для чанков. 0..1
                content = doc.page_content
                response = f'\n=====================Отрывок документа №{i + 1}=====================\n{content}\n'
                print(f'\n=====================Отрывок документа №{i + 1}=====================\n')
                print(f'=== score = {score}  Metadata документа ------------ {doc.metadata}')
                print(f'\n{content}\n')
                responses.append(response)


        messages = [
            {"role": "system", "content": prompt},
            {"role": "user",
             "content": f"Документ с информацией для ответа пользователю: {responses}\n\nВопрос клиента: \n{input_text}"}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=temp
        )

        answer = completion.choices[0].message.content

        # Добавляем вопрос пользователя и ответ системы в историю
        HISTORY.append((topic, answer if answer is not None else ''))

        return insert_newlines(answer)  # возвращает ответ


In [ ]:
def run_dialog(system_doc_url, knowledge_base_url):
    topic = 'Приветствую, я нейро-помощник KIA. Подскажите, какая у Вас марка машины? Как я могу к Вам обращаться?'
    print('\nМенеджер: ', topic+'\n\n')
    user_question = input('\nКлиент: ')
    answer = answer_index(knowledge_base_url, user_question)
    HISTORY.append((topic, answer if answer is not None else ''))
    print('\nМенеджер: ', 'Какая Вам требуется помощь?'+'\n\n')
    while True:
        user_question = input('\nКлиент: ')
        if ((user_question.lower() == 'stop') or (user_question.lower() == 'стоп')):
            break
        answer = answer_index(knowledge_base_url, user_question)
        print('\nМенеджер: ', answer+'\n\n')

    return

In [ ]:
# Промпт моделей машин
prompt = load_document_text ('https://docs.google.com/document/d/1i8HA7cX4Ut-tb9rf8wOgERU7lLe66xJYscizGtSSJl0')

In [ ]:
temperature=0.3
verbose=0
#HISTORY = [] # Убери решетку если нужно обновлять историю диалогов после запуска, если не нужно обновлять то оставь так

run_dialog(prompt, knowledge_only_langchain)


Менеджер:  Приветствую, я нейро-помощник KIA. Подскажите, какая у Вас марка машины? Как я могу к Вам обращаться?



=====================Отрывок документа №1=====================

=== score = 0.23519188165664673  Metadata документа ------------ {'Header 1': 'models', 'Header 2': 'Kia Carnival 2022', 'Header 3': 'Территория стиля и комфорта'}

Территория стиля и комфорта
Что перед вами? Роскошный и брутальный кроссовер? Солидный и практичный семейный автомобиль? Высокотехнологичное средство передвижения или просторный и комфортный пати-вэн для большой компании? Kia Carnival совмещает все это и готов удивлять вас долгие годы.
Фото https://cdn.kia.ru/resize/1920x960/media-data/infographic/file/b32628c8-1d7d-401c-a57a-1169c5700dcf.jpeg


=====================Отрывок документа №2=====================

=== score = 0.2443714141845703  Metadata документа ------------ {'Header 1': 'models', 'Header 2': 'Kia Carnival 2022', 'Header 3': 'Комплектации', 'Header 4': 'Комплектация: Kia Carnival Pre